### Pyterrier test for hackaton

In [2]:
!pip install datasets

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
   ---------------------------------------- 0.0/485.4 kB ? eta -:--:--
   ---------- ----------------------------- 122.9/485.4 kB 3.6 MB/s eta 0:00:01
   -------------------------------------- - 471.0/485.4 kB 5.9 MB/s eta 0:00:01
   ---------------------------------------- 485.4/485.4 kB 5.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/116.3 kB ? eta -:--:--
   ---------------------------------------- 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/468.0 kB ? eta -:--:--
   --------------------------------------- 468.0/468.0 kB 14.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/134.8 kB ? eta -:--:--
   ---------------------------------------- 134.8/134.8 kB 8.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/25.3 MB ? eta -:--:--
   -- ------------------------------------- 1.3/25.3 MB 41.3 MB/s eta 0:00:01
   ----- ------------------

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlflow 2.5.0 requires pyarrow<13,>=4.0.0, but you have pyarrow 19.0.1 which is incompatible.


In [1]:
import pyterrier as pt
import pandas as pd
from datasets import load_dataset

if not pt.started():
    pt.init()

# Load the dataset from Hugging Face
dataset = load_dataset("AGBonnet/augmented-clinical-notes", split="train")



C:\Users\david\AppData\Local\Temp\ipykernel_30492\4123888700.py:5: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  if not pt.started():
Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]
C:\Users\david\AppData\Local\Temp\ipykernel_30492\4123888700.py:6: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
java is now started automatically with default settings. To force initialisation early, run:
pt.java.init() # optional, forces java initialisation
  pt.init()


In [2]:
dataset

Dataset({
    features: ['idx', 'note', 'full_note', 'conversation', 'summary'],
    num_rows: 30000
})

In [3]:
# Choose the 'full_note' field for indexing.
# Alternatively, you could use 'note' or combine fields if that better fits your use case.
docs = [{'docno': row['idx'], 'text': row['full_note']} for row in dataset]

# Create a DataFrame from the records.
df = pd.DataFrame(docs)
df.describe()

# # Index the documents using PyTerrier's IterDictIndexer.
# indexer = pt.index.IterDictIndexer("./pt_index")
# index_ref = indexer.index(df.to_dict(orient='records'), fields=['docno', 'text'])

# indexer = pt.index.IterDictIndexer("./pt_index", fields=["docno", "text"], text_attrs="text")
# index_ref = indexer.index(df.to_dict(orient='records'))

,docno,text
count,30000,30000
unique,30000,24341
top,155216,A 54-year-old Caucasian female proceeded to ou...
freq,1,10


In [6]:
import os
import shutil

# Define an absolute index path (adjust this to a valid writable location)
index_path = r"D:\SRC\DS@GT\eRisk25\eRisk25-datasets\pt_index"

# ✅ Delete existing index if needed
if os.path.exists(index_path):
    shutil.rmtree(index_path)
    print(f"✅ Deleted existing index at: {index_path}")    


indexer = pt.index.IterDictIndexer(index_path)
index_ref = indexer.index(df.to_dict(orient='records'))

✅ Deleted existing index at: D:\SRC\DS@GT\eRisk25\eRisk25-datasets\pt_index


In [7]:
# Initialize BM25 retrieval model.
bm25 = pt.BatchRetrieve(index_ref, wmodel="BM25")

C:\Users\david\AppData\Local\Temp\ipykernel_30492\2155857068.py:2: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  bm25 = pt.BatchRetrieve(index_ref, wmodel="BM25")


In [8]:
from pyterrier_t5 import MonoT5ReRanker
monoT5 = MonoT5ReRanker()

C:\Users\david\.conda\envs\eriskenv\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [10]:
pipeline = bm25 >> monoT5

In [11]:
query = "patients with hypertension"
results = pipeline.search(query)
print(results.head(10))

KeyError: 'text'

This error occurs because the reranker expects the input DataFrame to include a column named "text", but your retrieval results don't have it. When BM25 retrieves documents from your index, it doesn't automatically include the full document text unless you explicitly request it via the metadata parameter.

To fix this, update your BM25 initialization to include the "text" field. For example:

python
Copy
bm25 = pt.BatchRetrieve(index_ref, wmodel="BM25", metadata=["text"])
This tells BM25 to attach the "text" field (i.e., the full clinical note) from your index to each retrieved document. With this change, the subsequent reranker (which expects a "text" column) should work correctly.

In [12]:
bm25 = pt.BatchRetrieve(index_ref, wmodel="BM25", metadata=["text"])

C:\Users\david\AppData\Local\Temp\ipykernel_30492\1657241519.py:1: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  bm25 = pt.BatchRetrieve(index_ref, wmodel="BM25", metadata=["text"])


In [14]:
# Build the two-stage pipeline.
pipeline = bm25 >> monoT5

# Run a query.
query = "patients with hypertension"
results = pipeline.search(query)
print(results.head(10))

monoT5: 100%|███████████████████████████████████████████████████████████████████| 250/250 [00:25<00:00,  9.81batches/s]

   qid  docid text                       query      score  rank
1    1    350       patients with hypertension -11.283523     0
3    1   8847       patients with hypertension -11.283523     1
5    1  14075       patients with hypertension -11.283523     2
7    1  19041       patients with hypertension -11.283523     3
9    1  19248       patients with hypertension -11.283523     4
11   1  20468       patients with hypertension -11.283523     5
13   1  27145       patients with hypertension -11.283523     6
15   1  17649       patients with hypertension -11.283523     7
17   1  11276       patients with hypertension -11.283523     8
19   1  23561       patients with hypertension -11.283523     9
